In [1]:
import requests
import json
import csv
import time
from urllib.parse import urlparse
import os

In [2]:
csv_output_file = './data/crawl/tiktok_comments.csv'
labeled_csv_output_file = './data/crawl/tiktok_comments_labeled.csv'

In [3]:
def label_emotion_with_lmstudio(text):
    """
    Phân loại cảm xúc văn bản tiếng Việt dùng LM Studio (local LLM) chỉ dựa trên text và teen code
    0: Vui vẻ, 1: Tức giận, 2: Buồn, 3: Sợ hãi, 4: Trung lập
    """
    try:
        # Validate input
        if not text or not isinstance(text, str):
            print("⚠️ Input không hợp lệ")
            return 4

        # Từ điển teen code phổ biến và cảm xúc tương ứng
        teen_code_dict = {
            # Vui vẻ
            "vk": 0, "vler": 0, "qtqd": 0, "ckđ": 0, "hehe": 0, "hihi": 0, "vcl": 0, "vl": 0,
            # Tức giận
            "dm": 1, "dmm": 1, "clm": 1, "clmm": 1, "dkm": 1, "dkmm": 1, "dcm": 1, "dcmm": 1, "lạy": 1,
            # Buồn
            "tđn": 2, "sml": 2, "cc": 2, "oz": 2, "tđb": 2, "tđx": 2,
            # sợ hãi
            "hãi": 3, "hú hồn": 3, "vl": 3, "bay màu": 3, "vãi đạn": 3
        }

        # Kiểm tra teen code (chuyển sang chữ thường để so sánh)
        lower_text = text.lower().strip()
        if lower_text in teen_code_dict:
            return teen_code_dict[lower_text]

        # Optimized prompt chỉ tập trung vào text và teen code
        prompt = """Bạn là chuyên gia phân loại cảm xúc và Hãy phân loại cảm xúc của 1 đoạn comment TIẾNG VIỆT sau thành 1 trong 5 số (chỉ dựa trên văn bản):
0: Vui vẻ - niềm vui, hạnh phúc, tích cực
   • Teen code: vk, vler, qtqd, ckđ, hehe, hihi
   VD: "Thật tuyệt vời", "Hôm nay là một ngày vui", "vk quá đi"
1: Tức giận - giận dữ, khó chịu, bực bội
   • Teen code: dm, dmm, clm, dkm
   VD: "Đồ ngu ngốc!", "Tao cực kỳ tức giận", "clm mày"
2: Buồn - nỗi buồn, thất vọng, đau khổ
   • Teen code: tđn, sml, oz, tđb
   VD: "Cuộc sống thật chán", "Hôm nay buồn quá", "sml rồi"
3: sợ hãi - lo âu (cảm xúc mà con người cảm nhận được khi cảm thấy mối đe doạ, nguy hiểm hoặc rủi ro, dù thật hay tưởng tượng)
   • Teen code: hãi vl, hú hồn chim én, sợ vl, thấy mà sợ
   VD: "Nghe kể chuyện ma xong mà sợ vl luôn á", "Nhìn cái mặt ông thầy lúc gọi tên tui mà hãi vl", ...
4: Trung lập - không cảm xúc rõ ràng
   VD: "Hôm nay trời mưa", "Tôi đi làm lúc 8 giờ", "Đây là thông tin khách quan"

Bình luận cần phân loại: "{}"
→ Chỉ trả về 1 số từ 0-4, không giải thích:""".format(text)

        payload = {
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.3,
            "max_tokens": 1,
            "stop": ["\n"]
        }

        # Gọi API LM Studio
        response = requests.post(
            "http://localhost:1234/v1/chat/completions",
            headers={'Content-Type': 'application/json'},
            json=payload,
            timeout=600
        )

        # Xử lý response
        response.raise_for_status()
        result = response.json()
        
        # Lấy kết quả (đảm bảo chỉ lấy số)
        output = result["choices"][0]["message"]["content"].strip()
        
        if output.isdigit() and 0 <= int(output) <= 4:
            return int(output)
            
        print(f"⚠️ Kết quả không hợp lệ: '{output}' cho: '{text[:30]}...'")
        return 4
    
    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi kết nối LM Studio: '{text[:30]}...' | Lỗi: {str(e)}")
        return 4
    except Exception as e:
        print(f"❌ Lỗi xử lý: '{text[:30]}...' | Lỗi: {type(e).__name__}: {str(e)}")
        return 4

In [4]:
def label_and_export_comments(input_csv_file, output_csv_file, checkpoint_interval=10):
    import csv, os, time

    labeled_comments = []
    processed_lines = set()

    # Nếu file output đã tồn tại, đọc các dòng đã xử lý
    if os.path.exists(output_csv_file):
        with open(output_csv_file, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if 'line_number' in row:
                    processed_lines.add(int(row['line_number']))

    print(f"🔄 Tiếp tục gán nhãn. Đã gán: {len(processed_lines)} comments")

    try:
        with open(input_csv_file, 'r', encoding='utf-8') as csvfile:
            all_lines = list(csvfile.readlines())

        header = all_lines[0].strip().split(',')  # get column names
        reader = csv.DictReader(all_lines[1:], fieldnames=header)
        batch = []

        for line_number, row in enumerate(reader, start=2):  # start=2 vì dòng 1 là header
            if line_number in processed_lines:
                continue

            text = row.get('text', '').strip()
            if not text:
                continue

            # Gán nhãn cảm xúc
            emotion_label = label_emotion_with_lmstudio(text)
            labeled_comment = {
                'line_number': line_number,
                'text': text,
                'emotion_label': emotion_label
            }

            batch.append(labeled_comment)
            processed_lines.add(line_number)

            print(f"Line {line_number} | Labeled: {text[:50]}... → {emotion_label}")

            # Lưu checkpoint
            if len(batch) >= checkpoint_interval:
                save_to_csv(output_csv_file, batch)
                print(f"💾 Checkpoint saved {len(batch)} comments (Total: {len(processed_lines)})")
                batch.clear()

            time.sleep(3)  # tránh bị rate limit

        # Lưu phần còn lại
        if batch:
            save_to_csv(output_csv_file, batch)
            print(f"✅ Final save {len(batch)} comments (Total: {len(processed_lines)})")

    except Exception as e:
        print(f"\n❌ Error during labeling: {str(e)}")


def save_to_csv(file_path, data, fieldnames=['line_number', 'text', 'emotion_label']):
    file_exists = os.path.exists(file_path)
    with open(file_path, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerows(data)


### Phân loại cảm xúc
0: Vui vẻ, 1: Tức giận, 2: Buồn bã, 3: Sợ hãi, 4: Trung lập.

In [5]:
label_and_export_comments(csv_output_file, labeled_csv_output_file, checkpoint_interval=5)

🔄 Tiếp tục gán nhãn. Đã gán: 8120 comments
Line 8122 | Labeled: Nhà tôi sẽ ngược lại nha. T để cho ck đi ở công kh... → 2
Line 8123 | Labeled: 😁😁😁... → 0
Line 8124 | Labeled: chị vợ ko cần làm gì 2 đứa tự động lồi ra... → 0
Line 8125 | Labeled: 😁😁😁... → 0
Line 8126 | Labeled: tr tr thiệt luôn hả 🤣... → 0
💾 Checkpoint saved 5 comments (Total: 8125)
Line 8127 | Labeled: đúng là miền Nam có những vụ án chỉ có trong tưởng... → 4
Line 8128 | Labeled: Sợ thế! thế này ai giám cặp bồ nữa... → 3
Line 8129 | Labeled: Sao giờ tình yêu nó lạ dị... → 2
Line 8130 | Labeled: 😁... → 0
Line 8131 | Labeled: Có bản lĩnh... → 0
💾 Checkpoint saved 5 comments (Total: 8130)
Line 8132 | Labeled: ok đó Ah 🤣🤣🤣... → 0
Line 8133 | Labeled: Đàn bà ngoại tình vì cảm xúc, còn đàn ông thì vì t... → 4
Line 8134 | Labeled: thế bị tịch thu xe àh... → 4
Line 8135 | Labeled: Nhung@Mẹ Bin Jin... → 4
Line 8136 | Labeled: Mấy ông ngoại tình vào đây hùa đây mà... → 1
💾 Checkpoint saved 5 comments (Total: 8135)
Line 8137 | Lab

KeyboardInterrupt: 